import pandas as pd
import numpy as np 
import sqlite3

In [2]:
df = pd.read_csv(r"ENTRADASEBZMORUMBI2025 - Entradas.csv")

## Leitura e primeiras colunas deletadas

In [102]:
df = pd.read_csv(r"ENTRADASEBZMORUMBI2025 - Entradas.csv")
df = df.dropna(subset=("Colaborador"))
df.drop(columns=["Unnamed: 48","Unnamed: 49"],inplace=True) 
df.rename(columns={' ': "data"}, inplace=True)

### Definindo classe e funções de limpeza

In [106]:
from typing import Dict, Union
import pandas as pd
import numpy as np

passage: Dict[str, pd.DataFrame] = {}

class data_cleaning(pd.DataFrame):
    
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
    def to_date(self, date_column: str) -> "data_cleaning":
        """
        Converts a column to datetime format.

        Args:
            date_column (str): The name of the column to convert.

        Returns:
            data_cleaning: The updated DataFrame with the column converted to datetime.
        """
        self[date_column] = pd.to_datetime(self[date_column], format="%d/%m/%Y")
        return self
        
    def normalize_dollar(self, dollar_column: str) -> "data_cleaning":
        """
        Normalizes a column with dollar values by removing currency symbols and converting to float.

        Args:
            dollar_column (str): The name of the column to normalize.

        Returns:
            data_cleaning: The updated DataFrame with the column normalized.
        """
        if dollar_column in self.columns:
            self[dollar_column] = self[dollar_column].apply(
                lambda x: x.replace("R$ ", "").replace(",", ".").replace(" ", "").replace("0.0.0", "0") if isinstance(x, str) else x
            )
            self[dollar_column] = self[dollar_column].astype(float)
        return self
    
    def to_boolean(self, column: str) -> "data_cleaning":
        """
        Converts a column with "SIM"/"NÃO" values to boolean.

        Args:
            column (str): The name of the column to convert.

        Returns:
            data_cleaning: The updated DataFrame with the column converted to boolean.
        """
        if column in self.columns:
            self[column] = self[column].map({"NÃO": False, "SIM": True}).astype(bool)
        return self   
    
    def to_null(self, column: str) -> "data_cleaning":
        """
        Replaces "VAZIO" values in a column with NaN.

        Args:
            column (str): The name of the column to process.

        Returns:
            data_cleaning: The updated DataFrame with "VAZIO" replaced by NaN.
        """
        if column in self.columns:
            self[column] = self[column].apply(lambda x: np.nan if x == "VAZIO" else x)
        return self
    
    def rename_second_column(self) -> "data_cleaning":
        """
        Renames the first four columns of the DataFrame to standardized names.

        Returns:
            data_cleaning: The updated DataFrame with renamed columns.
        """
        if len(self.columns) > 1:
            self.rename(columns={self.columns[0]: "produto"}, inplace=True)
            self.rename(columns={self.columns[1]: "vl_venda"}, inplace=True)
            self.rename(columns={self.columns[2]: "compra"}, inplace=True)
            self.rename(columns={self.columns[3]: "lucro"}, inplace=True)
        return self
    
    @staticmethod
    def to_its_own_dimension(base_dataframe: pd.DataFrame, key: str, value: str) -> pd.DataFrame:
        """
        Creates a new DataFrame with unique combinations of key and value columns.

        Args:
            base_dataframe (pd.DataFrame): The base DataFrame.
            key (str): The column name to use as the key.
            value (str): The column name to use as the value.

        Returns:
            pd.DataFrame: The new DataFrame with unique combinations.
        """
        new_dataframe = base_dataframe[["ID", key, value]].drop_duplicates()
        new_dataframe["type"] = key 
        new_dataframe = new_dataframe.rename(columns={key: "produtos", value: "quantidade"})

        passage[key] = new_dataframe

        combined_dataframe = pd.concat(passage.values(), ignore_index=True)

        return combined_dataframe
    
class feature_engineering(data_cleaning):
     
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
         
    def index_values(self, column: str) -> "feature_engineering":
        """
        Creates a mapping of unique values in a column to unique indices.

        Args:
            column (str): The column to index.

        Returns:
            feature_engineering: The updated DataFrame with an indexed column.
        """
        unique_values = self[column].unique()
        dicionario = {value: key for key, value in enumerate(unique_values)}
        self[f"id_{column}"] = self[column].map(dicionario)
        return self




### Limpando dataframe

In [ ]:
df = data_cleaning(df)
df = df.to_date("data")

colunas_valor = [col for col in df.columns if col.startswith("Valor")] + ["TaxaMaquina","Total(S+P)","Total(S+P)*T","Total[(S+P)-LP]*T","TotalS+LP-Col","TotalColaborador","Colaborador50%"] + [col for col in df.columns if col.startswith("AuxValor")] + [col for col in df.columns if col.startswith("AuxDesconto")]
for col in colunas_valor:
    df = df.normalize_dollar(col)

colunas_valor = [col for col in df.columns if col.startswith("AuxValor")]
for col in colunas_valor:
    df = df.normalize_dollar(col)
    
df = df.to_boolean("ClienteNovo")

colunas = ["Produto","Doces","Salgados","Bebidas"]
for col in colunas:
    df = df.to_null(col)
    
combine_columns = {
    "Produto": "QuantidadeProduto",
    "Doces": "QuantidadeDoces",
    "Salgados": "QuantidadeSalgados",
    "Bebidas": "QuantidadeBebidas"
}

## Criando dimensão de produto

In [56]:
produtos = pd.read_csv(r"c:\Users\luiz\Documents\GitHub\EZmorumbi\Data Cleaning\ENTRADASEBZMORUMBI2025 - Produtos.csv")


#### Limpeza produtos

In [ ]:
dim_servico = produtos[["SERVIÇO","VALOR DO SERVIÇO"]]
dim_produto = produtos[["PRODUTO","VALOR DE VENDA","VALOR DE COMPRA","LUCRO"]]
dim_doce = produtos[["DOCE","VALOR DE VENDA.1","VALOR DE COMPRA.1","LUCRO.1"]]
dim_salgado = produtos[["SALGADO","VALOR DE VENDA.2","VALOR DE COMPRA.2","LUCRO.2"]]
dim_bebida = produtos[["BEBIDA","VALOR DE VENDA.3","VALOR DE COMPRA.3","LUCRO.3"]]
dim_funcionario = produtos[["FUNCIONARIO","PORCENTAGEM"]]
dim_taxa = produtos[["FUNCAO","TAXA"]]

dim_servico["tipo"] = "servico"
dim_doce["tipo"] = "doce"
dim_salgado["tipo"] = "salgado"
dim_bebida["tipo"] = "bebida"
dim_funcionario["tipo"] = "funcionario"
dim_taxa["tipo"] = "taxa"

dataframes = [dim_servico,dim_produto,dim_doce,dim_salgado,dim_bebida,dim_funcionario,dim_taxa]

dim_servico = data_cleaning(dim_servico)
dim_produto = data_cleaning(dim_produto)
dim_doce = data_cleaning(dim_doce)
dim_salgado = data_cleaning(dim_salgado)

dim_produto.rename_second_column()
dim_doce.rename_second_column()
dim_salgado.rename_second_column()
dim_bebida.rename_second_column()

for data in dataframes:
    data.dropna(inplace=True)
    
dim_servico.rename(columns={
    "SERVIÇO" : "servico",
    "VALOR DO SERVIÇO" : "vl_venda"
})

dim_produto = dim_produto.normalize_dollar("lucro")
dim_doce = dim_doce.normalize_dollar("lucro")
dim_salgado = dim_salgado.normalize_dollar("lucro")
dim_bebida = dim_bebida.normalize_dollar("lucro")
dim_produto = dim_produto.normalize_dollar("vl_venda")
dim_doce = dim_doce.normalize_dollar("vl_venda")
dim_salgado = dim_salgado.normalize_dollar("vl_venda")
dim_bebida = dim_bebida.normalize_dollar("vl_venda")
dim_produto = dim_produto.normalize_dollar("compra")
dim_doce = dim_doce.normalize_dollar("compra")
dim_salgado = dim_salgado.normalize_dollar("compra")
dim_bebida = dim_bebida.normalize_dollar("compra")

def to_string(dataframe,coluna):
    dataframe[coluna] = dataframe[coluna].astype(str)
    return dataframe[coluna].apply(lambda x: type(x)).value_counts()

to_string(dim_bebida,"produto")
to_string(dim_produto,"produto")
dim_produto = dim_produto[dim_produto["produto"] != "VAZIO"]

dataframes = [dim_bebida, dim_doce, dim_salgado]
for df in dataframes:
    df.dropna(subset=["vl_venda"],inplace=True)
    
dim_produto_2 = pd.concat([dim_bebida, dim_doce, dim_salgado])

dim_p_b = dim_produto.merge(dim_produto_2, on="produto", how="left", suffixes=("", "n_"))
dim_p_b.drop(columns=["vl_vendan_","compran_","lucron_"],inplace=True)
dim_p_b.dropna(subset="vl_venda",inplace=True)
dim_p_b["tipo"] = dim_p_b["tipo"].apply(lambda x: "barbearia" if pd.isna(x) else x)
dim_p_b.drop_duplicates(subset=["produto"],inplace=True)
dim_p_b = feature_engineering(dim_p_b)
dim_p_b = dim_p_b.index_values("produto")
produto_map = dim_p_b.set_index("produto")["id_produto"].to_dict()

passage: Dict[str, pd.DataFrame] = {}

for key, value in combine_columns.items():
    result_dataframe = data_cleaning.to_its_own_dimension(df, key, value)

result_dataframe.dropna(subset="produtos", inplace=True)
result_dataframe = result_dataframe.assign(produtos=result_dataframe["produtos"].str.split(",")).explode("produtos")
result_dataframe["quantidade"] = 1
result_dataframe["produtos"].unique()
fato_produtos = result_dataframe
fato_produtos.rename(columns={"ID": "id_servico"}, inplace=True)
fato_produtos["id_servico"] = fato_produtos["id_servico"].astype(int)
fato_produtos = fato_produtos.merge(dim_p_b, left_on="produtos", right_on="produto", how="left")



## Criando tabela Fato


In [ ]:
tabela_fato = new_df[['data', 'ID', 'Colaborador', 'Cliente', 'Servico', 'ValordoServico',
       'ClienteNovo', 'Local', 'Produto', 'QuantidadeProduto',
       'ValorTotaldosProduto', 'Doces',
       'QuantidadeDoces', 'ValorTotaldosDoces', 'Salgados', 'QuantidadeSalgados',
       'Bebidas', 'QuantidadeBebidas',
       'ValorTotaldasBebidas', 'FormadePagamento', 'TaxaMaquina', 'Total(S+P)',
       'Total(S+P)*T', 'Total[(S+P)-LP]*T', 'TotalS+LP-Col',
       'TotalColaborador', 'Colaborador50%']]

tabela_fato = tabela_fato.rename(columns={
    'data': 'data',
    'ID': 'id',
    'Colaborador': 'colaborador',
    'Cliente': 'cliente',
    'Servico': 'servico',
    'ValordoServico': 'vl_servico',
    'ClienteNovo': 'cliente_novo',
    'Local': 'local',  
    'Produto': 'produto',
    'QuantidadeProduto': 'qtd_produto',
    'ValorTotaldosProduto': 'vl_total_produto',
    'Doces': 'doce',
    'QuantidadeDoces': 'qtd_doce',
    'ValorTotaldosDoces': 'vl_total_doces',
    'Salgados': 'salgado',  
    'QuantidadeSalgados': 'qtd_salgado',
    'ValorTotaldosSalgados': 'vl_total_salgado',
    'Bebidas': 'bebida',  
    'QuantidadeBebidas': 'qtd_bebida',
    'ValorTotaldasBebidas': 'vl_total_bebida',
    'FormadePagamento': 'forma_de_pagamento',
    'TaxaMaquina': 'tx_maquina',
    'Total(S+P)': 'total_sp',
    'Total(S+P)*T': 'total_spt',
    'Total[(S+P)-LP]*T': 'total_splt',
    'TotalS+LP-Col': 'total_slp_col',
    'TotalColaborador': 'total_colaborador',
    'Colaborador50%': 'colaborador_50'
    'local': 'fidelizado'
})


## Criação da tabela 

In [ ]:
def create_table(cursor):
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tabela_fato (
            data DATETIME,
            id DOUBLE,
            colaborador TEXT,
            cliente TEXT,
            servico TEXT,
            vl_servico DOUBLE,
            cliente_novo BOOLEAN,
            fidelizado TEXT,
            produto TEXT,
            qtd_produto TEXT,
            vl_total_produto DOUBLE,
            doce TEXT,
            qtd_doce DOUBLE,
            vl_total_doces DOUBLE,
            salgado TEXT,
            qtd_salgado DOUBLE,
            bebida TEXT,
            qtd_bebida TEXT,
            vl_total_bebida DOUBLE,
            forma_de_pagamento TEXT,
            tx_maquina DOUBLE,
            total_sp DOUBLE,
            total_spt DOUBLE,
            total_splt DOUBLE,
            total_slp_col DOUBLE,
            total_colaborador DOUBLE,
            colaborador_50 DOUBLE
        )
    ''')

def insert_data(cursor, df):
    for _, row in df.iterrows():
        cursor.execute('''
            INSERT INTO tabela_fato VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''', tuple(row))

def main(df):
    conn = mysql.connector.connect(
        host='database-atinova.ct6oomqu6y49.sa-east-1.rds.amazonaws.com',
        user='integrantes',
        password='grupoPI2025',
        database='barbearia',  # Substituir pelo nome correto do banco de dados
        port=3306
    )
    cursor = conn.cursor()
    
    create_table(cursor)
    
    df.fillna('', inplace=True)  # Substituir NaN por strings vazias
    insert_data(cursor, df)
    
    conn.commit()
    cursor.close()
    conn.close()

# Chame a função passando seu DataFrame diretamente
main(tabela_fato)
